# Import Modules
Have to restart kernel after installing DeepStack

In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import load_model as kerasload
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys

af_dir = '../../all_faces_bucket/'
disk_data_dir = '../../all_faces_disk/home/jupyter/forensics_split/'
!pip install git+https://github.com/qubvel/efficientnet

sys.path.insert(1, '/home/jupyter/DeepFake-2019-20/predictions/DeepStack/deepstack')
from base import *
from ensemble import *
import efficientnet.tfkeras
# from DeepStack.base import KerasMember
# from deepstack.ensemble import r
# from deepstack.ensemble import DirichletEnsemble
# from deepstack.ensemble import StackEnsemble
# from deepstack.base import Member
import joblib
import os
import glob
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
# !sudo kill -9 PID # clear GPU memory where 9 is PID number
# !watch -n0.1 nvidia-smi

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-z17z0_ah
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-z17z0_ah
  Created wheel for efficientnet: filename=efficientnet-1.1.0-py3-none-any.whl size=18397 sha256=9fb61f3edff1fac7bd54d76ef75c5bdbd224927615ec47f138d2c2d49168772b
  Stored in directory: /tmp/pip-ephem-wheel-cache-8718n97i/wheels/11/69/85/814d64d694c96db0eef17b718042d644a1e54f113920481920
Successfully built efficientnet


# Load models

In [2]:
def get_model(config):
    '''Loads one of the saved models based on specified config'''
    
    return kerasload(af_dir + 'trained_models/saved_models/config' + str(config) + '.h5')

model1 = get_model(2)
model2 = get_model(3)
# model3 = get_model(2)
# model4 = get_model(3)
# model5 = get_model(4)
# model6 = get_model(5)
# model7 = get_model(6)
# model8 = get_model(7)
# model9 = get_model(8)
# model10 = get_model(9)
# model11 = get_model(10)
# model11 = get_model(11)

# Create image generators

In [9]:
# Create data generators
def augment_data(directory, batch):
    '''Prepares train-time augmentation using given training and validations data)
    
    Returns train_data, val_data'''

    datagen_train = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=True,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=True,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            brightness_range=None,
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
#             preprocessing_function=deepaug.joint_function,
            # image data format, either "channels_first" or "channels_last"
            data_format=None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            )
    
    datagen_test = ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True)

    # Classes give the folders storing the two different categories
    train_data = datagen_train.flow_from_directory(directory + '/train',
                                             target_size=(224,224), batch_size = batch, classes=['authentic','fake'])
    
    val_data = datagen_test.flow_from_directory(directory + '/validation',
                                             target_size=(224,224), batch_size = batch, classes=['authentic','fake'])
    
    return train_data, val_data

train_data, val_data = augment_data('../../forensics_split', batch=128)

Found 113928 images belonging to 2 classes.
Found 21291 images belonging to 2 classes.


# Instantiate KerasMembers
This will find all training and validation predictions. A fit method can then be applied to obtain to the members to create the ensemble.

In [10]:
member1 = KerasMember(name="model1", keras_model=model1, train_batches=train_data, val_batches=val_data)
member2 = KerasMember(name="model2", keras_model=model2, train_batches=train_data, val_batches=val_data)
# member3 = KerasMember(name="model3", keras_model=model3, train_batches=train_data, val_batches=val_data)
# member4 = KerasMember(name="model4", keras_model=model4, train_batches=train_data, val_batches=val_data)

167/167 [==============================] - 48s 287ms/step


In [ ]:
# Load members
folder = '../../all_faces_bucket/trained_models/saved_models/stack'
stack = joblib.load(os.path.join(folder, "stackensemble.joblib"))
stack.members = []
if folder[-1] != os.sep:
    folder += os.sep
for fn in glob.glob(folder + "**/"):
    member = Member.load(fn)
    stack.add_member(member)
    
member1 = stack.members[0]
member2 = stack.members[1]

# Fit weighted average ensemble

In [24]:
# wAvgEnsemble = DirichletEnsemble()
# wAvgEnsemble.add_members([member1, member2])
# wAvgEnsemble.fit()
# wAvgEnsemble.describe()
wAvgEnsemble.bestweights

array([0.46218966, 0.53781034])

# Fit stacking ensemble

In [14]:
stack = StackEnsemble()
stack.model = RandomForestRegressor(verbose=0, n_estimators=200, 
                                  max_depth=15, n_jobs=20, min_samples_split=20)
stack.add_members([member1, member2])
stack.fit()
stack.describe()

Calling predict
[[0.9857336  0.01426641]
 [0.9958896  0.00411043]
 [0.99520314 0.00479682]
 ...
 [0.1587801  0.8412199 ]
 [0.01335015 0.98664993]
 [0.01235618 0.98764384]]
[[0.9857336  0.01426641]
 [0.9958896  0.00411043]
 [0.99520314 0.00479682]
 ...
 [0.1587801  0.8412199 ]
 [0.01335015 0.98664993]
 [0.01235618 0.98764384]]
model1 - roc_auc_score: 0.9938
[[9.4082749e-01 5.9172526e-02]
 [9.9677074e-01 3.2293212e-03]
 [9.9948561e-01 5.1431154e-04]
 ...
 [7.1437165e-02 9.2856288e-01]
 [1.4989050e-02 9.8501092e-01]
 [8.5764965e-03 9.9142355e-01]]
[[9.4082749e-01 5.9172526e-02]
 [9.9677074e-01 3.2293212e-03]
 [9.9948561e-01 5.1431154e-04]
 ...
 [7.1437165e-02 9.2856288e-01]
 [1.4989050e-02 9.8501092e-01]
 [8.5764965e-03 9.9142355e-01]]
model2 - roc_auc_score: 0.9926
[2.22591130e-03 1.03571757e-04 2.66549535e-04 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]
StackEnsemble roc_auc_score: 0.9914


0.99137755331052

In [11]:
stack = StackEnsemble()
stack.model = RandomForestRegressor(verbose=0, n_estimators=200, 
                                  max_depth=15, n_jobs=20, min_samples_split=20)
stack.add_members([member1, member2])
stack.fit()
stack.describe(metric='acc')

Calling predict
model1 - Test Accuracy: 0.9772
model2 - Test Accuracy: 0.9592
StackEnsemble Test Accuracy: 0.9770


0.9769855807618243

In [13]:
stack.predictions

(21291,)

# 3 Level Stacking

In [11]:
stack3 = StackEnsemble()

# 2nd Level Meta-Learner
estimators = [
    ('rf', RandomForestClassifier(verbose=0, n_estimators=200, max_depth=15, n_jobs=20, min_samples_split=30)),
    ('etr', ExtraTreesClassifier(verbose=0, n_estimators=200, max_depth=10, n_jobs=20, min_samples_split=20))
]
# 3rd Level Meta-Learner
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

stack3.model = clf
stack3.add_members([member1, member2])
stack3.fit()
stack3.describe()

Calling predict
model1 - roc_auc_score: 0.9918
model2 - roc_auc_score: 0.9870
StackEnsemble roc_auc_score: 0.9584


0.9584082769101238

In [12]:
# Save models
# wAvgEnsemble.save(af_dir + 'trained_models/saved_models/weighted_average')
stack.save(af_dir + 'trained_models/saved_models/stack')
stack3.save(af_dir + 'trained_models/saved_models/stack3')

<StackEnsemble: [model1, model2]>